In [1]:
# 三立新聞網SETN
# 爬蟲套件
import requests
from bs4 import BeautifulSoup
import json
from fake_useragent import UserAgent
# 資料整理類套件
import pandas as pd
import re
import time
import random
from tomorrow import threads
from pymongo import MongoClient
from datetime import datetime
import pymongo

In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [3]:
def GetAllLinks():
    """表層links"""
    links = []
    page = 1
    while True:
        url = "https://www.setn.com/ViewAll.aspx?PageGroupID=6&p="+str(page)
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text)
        if soup.find("div",class_="row NewsList").find_all("a",class_="gt") == []:
            break
#         if page > 5:
#             break
        for i in soup.find("div",class_="row NewsList").find_all("a",class_="gt"):
            links.append("https://www.setn.com/"+i.get("href"))
        page += 1
        time.sleep(random.randint(5,10))
    return links

In [4]:
def GetSETN(response):
    soup = BeautifulSoup(response.text)
    Url = soup.find("meta",property = "og:url").get("content")
    Author = soup.find("div",id = "Content1").find("p").text
    Title = soup.find("h1",class_="news-title-3").text
    CreateTime = soup.find("time",class_ = "page-date").text
    CreateTime = datetime.strptime(CreateTime,'%Y/%m/%d %H:%M:%S')
    Content_list = []
    for i in soup.find("div",id = "Content1").find_all("p")[1:]:
        Content_list.append(i.text.replace("\xa0",""))
    Content = "".join(Content_list)
    Tag = []
    for i in soup.find("div",class_ = "keyword page-keyword-area").find_all("li"):
        Tag.append(i.text)
     
    ndf = pd.DataFrame(data = [{'Source':'News',         
                                'Site':'三立新聞網',
                                'Url':Url,
                                'Author':Author,
                                'Title':Title,
                                'CreateTime':CreateTime,
                                'Content':Content,
                                'Tag':Tag}],
                       columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Tag']) 
    print(Url)
    time.sleep(random.randint(5,10))
    return ndf

In [5]:
#測試抓取一篇文章並存為是否成功
# testurl = 'https://www.setn.com/News.aspx?NewsID=639769'   #自定義
# resp = requests.get(testurl,headers=headers)
# GetSETN(resp)

In [6]:
# 多線程爬蟲定義
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass


In [7]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetSETN(response)
            allData.append(ndf)
        except:
            pass

    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    
    return df

In [8]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")


In [9]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
        

In [10]:
def dataframe_to_mongo_political(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
     

In [1]:
links = GetAllLinks()
success_count = 0
error_count = 0
allData = []
for link in links:
    try:
        res = requests.get(link, headers=headers)
        ndf = GetSETN(res)
        allData.append(ndf)
        success_count += 1
    except:
        error_count += 1
        pass
print(success_count)
print(error_count)
df = pd.concat(allData, ignore_index=True)

In [ ]:
dataframe_to_mongo_political(df)